In [121]:
def sigmoid(prediction):
    '''
    Sigmoid activation function
    '''
    return 1. / (1. + np.exp(-prediction))

In [122]:
def relu_(prediction):
    '''
    Relu activation function
    '''
    return prediction

In [123]:
def cost_mse_ols(design, data, beta):
    '''
    Mean squared error
    '''
    return (data - design.dot(beta)).T*(data - design.dot(beta))

In [124]:
def cost_grad_ols(design, data, beta):
    '''
    Calculates the first derivative of MSE w.r.t beta.
    '''
    return (2/len(data))*design.T.dot(design.dot(beta)-data) #logistic regression slides

In [125]:
def cost_log_ols(design, data, beta):
    '''
    Logisitic regression cost function
    '''
    return -data.dot(np.log(prediction)+1e-10) - ((1-data).dot(np.log(1-prediction + 1e-10)))

In [126]:
def cost_grad_log_ols(design, data, p):
    '''
    Gradient w.r.t log
    '''
    return (1/len(data))*design.T.dot(data-p)

In [127]:
def cost_mse_rid(design, data, beta, _lambda=1e-07):
    '''
    Mean squared error
    '''
    return (data - design.dot(beta)).T*(data - design.dot(beta)) + _lambda(np.sum(beta)**2)

In [128]:
def cost_grad_rid(design, data, beta, _lambda=1e-07):
    '''
    Calculates the first derivative of MSE w.r.t beta.
    '''
    regu_term = _lambda*np.sum(beta**2) 
    return (2/len(data))*design.T.dot(design.dot(beta)-data) + _lambda*np.sum(beta**2) + regu_term 

In [129]:
def cost_log_rid(design, data, beta, _lambda=1e-07):
    '''
    Logisitic regression cost function
    '''
    regu_term = _lambda*np.sum(beta**2) 
    return -data.dot(np.log(prediction)+1e-10) - ((1-data).dot(np.log(1-prediction + 1e-10))) + regu_term

In [130]:
def cost_grad_log_rid(design, data, p, beta, _lambda=1e-07):
    '''
    Gradient w.r.t log
    '''
    return (1/len(data))*design.T.dot(data-p) +2*_lambda*beta

In [131]:
def gradient_solver(N, eta, design, data, beta=None):
    M=len(data)
    if beta != None:
        beta = beta
    else:
        beta = np.random.randn(design.shape[1])
     
    for i in range(N):
        gradients = cost_grad_ols(design,frank,beta)
        beta -= eta*gradients
    return beta


In [132]:

import functions_class as fx
import classx as cl
import matplotlib.pyplot as plt
import numpy as np


n_x         = 50
x           = np.linspace(0, 1, n_x)
y           = np.linspace(0, 1, n_x)

x_mesh, y_mesh  = np.meshgrid(x,y)
noise_level     = 0.01
frank           = fx.FrankeFunction(x_mesh, y_mesh, noise_level)

frank = np.ravel(frank)



In [133]:
design = fx.DesignDesign(x,y,10)
data = frank.reshape([n_x*n_x,1])
np.random.seed(2018)
M=len(data)
N=10000
eta=0.1

beta = gradient_solver(N, eta, design, data)


prediction = design @ beta
pred = np.reshape(prediction,[n_x,n_x])



In [134]:
import pandas as pd
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score

# Trying to set the seed
np.random.seed(0)
import random
random.seed(0)

# Reading file into data frame
directory = os.getcwd()
filename = directory + '/cred_card.xls'
nanDict = {} # fjerner NaN 
dataframe = pd.read_excel(filename, header=1, skiprows=0, index_col=0, na_values=nanDict)


dataframe.rename(index=str, columns={"default payment next month": "defaultPaymentNextMonth"}, inplace=True)

# Features and targets 
X = dataframe.loc[:, dataframe.columns != 'defaultPaymentNextMonth'].values
y = dataframe.loc[:, dataframe.columns == 'defaultPaymentNextMonth'].values

# Categorical variables to one-hot's
onehotencoder = OneHotEncoder(categories="auto")

X = ColumnTransformer(
    [("", onehotencoder, [3]),],
    remainder="passthrough"
).fit_transform(X)



# Train-test split
trainingShare = 0.5 
seed  = 1
XTrain, XTest, yTrain, yTest=train_test_split(X, y, train_size=trainingShare, \
                                              test_size = 1-trainingShare,
                                             random_state=seed)

# Input Scaling
sc = StandardScaler()
XTrain = sc.fit_transform(XTrain)
XTest = sc.transform(XTest)

# One-hot's of the target vector
Y_train_onehot, Y_test_onehot = onehotencoder.fit_transform(yTrain), onehotencoder.fit_transform(yTest)


# Remove instances with zeros only for past bill statements or paid amounts
'''
dataframe = dataframe.drop(dataframe[(dataframe.BILL_AMT1 == 0) &
                (dataframe.BILL_AMT2 == 0) &
                (dataframe.BILL_AMT3 == 0) &
                (dataframe.BILL_AMT4 == 0) &
                (dataframe.BILL_AMT5 == 0) &
                (dataframe.BILL_AMT6 == 0) &
                (dataframe.PAY_AMT1 == 0) &
                (dataframe.PAY_AMT2 == 0) &
                (dataframe.PAY_AMT3 == 0) &
                (dataframe.PAY_AMT4 == 0) &
                (dataframe.PAY_AMT5 == 0) &
                (dataframe.PAY_AMT6 == 0)].index)
'''
dataframe = dataframe.drop(dataframe[(dataframe.BILL_AMT1 == 0) &
                (dataframe.BILL_AMT2 == 0) &
                (dataframe.BILL_AMT3 == 0) &
                (dataframe.BILL_AMT4 == 0) &
                (dataframe.BILL_AMT5 == 0) &
                (dataframe.BILL_AMT6 == 0)].index)

dataframe = dataframe.drop(dataframe[(dataframe.PAY_AMT1 == 0) &
                (dataframe.PAY_AMT2 == 0) &
                (dataframe.PAY_AMT3 == 0) &
                (dataframe.PAY_AMT4 == 0) &
                (dataframe.PAY_AMT5 == 0) &
                (dataframe.PAY_AMT6 == 0)].index)


In [135]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import GridSearchCV

#lambdas=np.logspace(-5,7,13)
#parameters = [{'C': 1./lambdas, "solver":["lbfgs"]}]#*len(parameters)}]
#scoring = ['accuracy', 'roc_auc']
#logReg = LogisticRegression()
#gridSearch = GridSearchCV(logReg, parameters, cv=5, scoring=scoring, refit='roc_auc')

In [136]:
from sklearn.linear_model import SGDClassifier
y=np.squeeze(y)
logreg = SGDClassifier(max_iter = 100000, penalty=None, eta0=0.1, learning_rate='constant' )
logreg.fit(X,y)
prediction = logreg.predict(X)


In [137]:
count = np.count_nonzero(y-prediction)

In [138]:
accuracy = (len(y)-count)/len(y)* 100
print(accuracy, '%') 

56.79 %


# Egen logistic regression.
Ulik prøving med dif deffinisjoner.. 

In [139]:
def sigmoid(x):
    # Activation function used to map any real value between 0 and 1
    return 1 / (1 + np.exp(-x))

def net_input(eta, x):
    # Computes the weighted sum of inputs
    return np.dot(x, eta)

def probability(eta, x):
    '''
    Returns the probability after passing through sigmoid
    '''
    return sigmoid(net_input(eta, x))


def cost_grad_ols(design, data, beta):
    '''
    Calculates the first derivative of MSE w.r.t beta.
    '''
    return (2/len(data))*design.T.dot(design.dot(beta)-data) #logistic regression slides

In [140]:
eta = 0.0001 # This is out eta
m = 10

Niteration = 100
beta = np.random.randn(26,1)
for iter in range(Niteration):
    sigmoid = 1/(1+np.exp(-(XTrain)@(beta))) # vi har ikke definert prediction i vår sigmoid definisjon. 
    gradients = -(np.transpose(XTrain)@(yTrain-sigmoid))
    beta -= eta*gradients
  
    # Cost function
    
    #total_cost = -(1 / m) * np.sum(y @ np.log(sigmoid(X))) + (1 - y) @ np.log(1 - sigmoid(X)))
    
    cost = -np.sum(np.transpose(yTrain)@np.log(sigmoid) + np.transpose(1-yTrain)@np.log(1-sigmoid))
    print('cost is', cost)

cost is nan
cost is inf
cost is inf
cost is inf
cost is inf
cost is inf
cost is 15246.062682755179
cost is 14081.252898542029
cost is 13226.557692386417
cost is 12574.541385602412
cost is 12060.231757364547
cost is 11644.483430307546
cost is 11301.443850104868
cost is 11013.469629338791
cost is 10768.735492955668
cost is 10559.611693112962
cost is 10381.463321176021
cost is 10231.745149905633
cost is 10109.121025145292
cost is 10012.370406455138
cost is 9939.260714541333
cost is 9885.963371712649
cost is 9847.644097653047
cost is 9819.857982081507
cost is 9799.359922388121
cost is 9784.006586555477
cost is 9772.378499999038
cost is 9763.487207875041
cost is 9756.610621817119
cost is 9751.209876967348
cost is 9746.884397208962
cost is 9743.341600851041
cost is 9740.372300618894
cost is 9737.829647949862
cost is 9735.611666729686
cost is 9733.647569860705
cost is 9731.887677153507
cost is 9730.296365702223
cost is 9728.847341407205
cost is 9727.520552704438
cost is 9726.300207595552
cost

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in matmul
  from ipykernel import kernelapp as app


cost is 9716.754901694712
cost is 9716.223294896077
cost is 9715.716749947838
cost is 9715.233165285537
cost is 9714.770649954986
cost is 9714.327500476371
cost is 9713.902180815468
cost is 9713.493304367177
cost is 9713.099618394695
cost is 9712.71998983806
cost is 9712.353393147423
cost is 9711.99889899861
cost is 9711.655664748258
cost is 9711.322925382023
cost is 9710.999986021427
cost is 9710.686214597232
cost is 9710.3810359799
cost is 9710.08392598957
cost is 9709.794406825295
cost is 9709.512042110171
cost is 9709.236433371163
cost is 9708.967215883236
cost is 9708.704056009154
cost is 9708.446647656656
cost is 9708.194710330703
cost is 9707.947986057767
cost is 9707.70623803433
cost is 9707.469247906547
cost is 9707.236814919077
cost is 9707.008753471418
cost is 9706.784892679183
cost is 9706.565074166105
cost is 9706.349151941527
cost is 9706.136990438472
cost is 9705.928464577459
cost is 9705.723458140761
cost is 9705.521863816884
cost is 9705.323582134433
cost is 9705.12852

# Accuracy. 
Både egen kode og tester med scikit. 

In [141]:
def predict(self, x):
    theta = parameters[:, np.newaxis]
    return probability(theta, x)

def accuracy(self, x, actual_classes, probab_threshold=0.5):
    predicted_classes = (predict(x) >= 
                         probab_threshold).astype(int)
    predicted_classes = predicted_classes.flatten()
    accuracy = np.mean(predicted_classes == actual_classes)
    return accuracy * 100
accuracy(X, y.flatten())

TypeError: accuracy() missing 1 required positional argument: 'actual_classes'

In [142]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 

model = LogisticRegression()
model.fit(X, y)
predicted_classes = model.predict(X)
accuracy = accuracy_score(y.flatten(),predicted_classes)
accuracy = accuracy * 100
parameters = model.coef_

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [143]:
#print(parameters)
print(accuracy, '%')

77.87333333333333 %


# Tips fra gruppelærer: 
Lage et enklere dataset som har x med 1000 elementer og første 500 verdiene = 0 og de siste etter = 1, slik at y fra 0 - 500 = 0 osv. Sjekker man logisitic regression på dette så vil accuracy være 100% med scikit. Kan bruke cost-funksjon med log for OLS. Ikke nødvenig å kjøre for Ridge og Lasso, dette er tidskrevende for oppgaven. Han tror ikke vi har tid til dette. Var inne på god tanke med loopen. 